In [6]:
# SKRYPT EKSPERYMENTALNY
# Uruchamia serię eksperymentów z różnymi parametrami algorytmu,
# zapisuje statystyki dla każdej kombinacji oraz mierzy pełny
# czas działania. Dane wynikowe trafiają do pliku CSV.
#
# Główne elementy:
#   - rozgrzanie kompilatora Numba (aby uniknąć narzutu JIT)
#   - wielokrotne uruchamianie algorytmu z tymi samymi parametrami
#   - wyznaczanie średnich kosztów i czasów
#   - zapisywanie najlepszej (minimalnej) trasy
#   - zapis wyników przy użyciu save_experiment_results()

%load_ext autoreload
%autoreload 2

import time
import pandas as pd
import numpy as np
from src.utils.tsp_loader import load_tsp_matrix
from src.algorithms.ihc_numba import solve_tsp
from src.utils.result_saver import save_experiment_results


# USTAWIENIA
TSP_FILES = ["Dane_TSP_76.xlsx"]

PARAM_GRID = {
    # "n_starts": [400],         # liczba restartów wspinaczki
    # "max_iter": [4000],        # liczba iteracji jednej wspinaczki
    # "stop_no_improve": [400],  # limit stagnacji
    # "neighborhood_type": ["two_opt"],
    "n_starts": [2000],         # liczba restartów wspinaczki
    "max_iter": [20000],        # liczba iteracji jednej wspinaczki
    "stop_no_improve": [2000],  # limit stagnacji
    "neighborhood_type": ["two_opt"],
}
# PARAM_GRID = {
#     "n_starts": [5, 10, 20, 50], # liczba losowych restartów wspinaczki
#     "max_iter": [100, 500, 1000], # maksymalna liczba iteracji jednej wspinaczki
#     "stop_no_improve": [10, 50, 100], # ile kroków bez poprawy kończy wspinaczkę
#     "neighborhood_type": ["swap", "insert", "two_opt"],
# }

REPEATS = 6   # liczba powtórzeń dla każdej kombinacji parametrów
results = []  # tablica na wyniki


# ROZGRZANIE NUMBA (KOMPILACJA JIT)
print("Rozgrzewanie Numba (kompilacja JIT)...")
D = load_tsp_matrix(TSP_FILES[0])
_ = solve_tsp(D, {"n_starts": 2})
print("Kompilacja zakończona.\n")


# GŁÓWNA PĘTLA
start_total = time.perf_counter()

for tsp_file in TSP_FILES:
    print(f"\nInstancja: {tsp_file}")
    D = load_tsp_matrix(tsp_file)

    for n_starts in PARAM_GRID["n_starts"]:
        for max_iter in PARAM_GRID["max_iter"]:
            for stop_no_improve in PARAM_GRID["stop_no_improve"]:
                for neighborhood_type in PARAM_GRID["neighborhood_type"]:

                    costs = []
                    runtimes = []
                    best_route_overall = None
                    min_cost = float("inf")

                    # wielokrotne powtórzenia
                    for _ in range(REPEATS):
                        params = {
                            "n_starts": n_starts,
                            "max_iter": max_iter,
                            "stop_no_improve": stop_no_improve,
                            "neighborhood_type": neighborhood_type,
                            "use_delta": True
                        }

                        start_t = time.perf_counter()
                        route, cost, _, _ = solve_tsp(D, params)
                        total_time = time.perf_counter() - start_t

                        costs.append(cost)
                        runtimes.append(total_time)

                        # zapamiętujemy najlepszą trasę z powtórzeń
                        if cost < min_cost:
                            min_cost = cost
                            best_route_overall = route

                    # zapis najlepszego route jako string
                    route_str = "-".join(map(str, best_route_overall))

                    # dodanie wiersza wyników
                    results.append({
                        "instance": tsp_file,
                        "n_starts": n_starts,
                        "max_iter": max_iter,
                        "stop_no_improve": stop_no_improve,
                        "neighborhood_type": neighborhood_type,
                        "mean_cost": round(np.mean(costs), 3),
                        "mean_runtime": np.mean(runtimes),
                        "min_cost": round(min_cost, 3),
                        "min_route": route_str,
                    })


# PODSUMOWANIE I ZAPIS WYNIKÓW
end_total = time.perf_counter()
elapsed = end_total - start_total

print(f"\nŁączny czas eksperymentów: {elapsed/60:.2f} min ({elapsed:.2f} sek)")

df = pd.DataFrame(results)
save_experiment_results(df, filename="with_delta__results.csv", time_seconds=int(elapsed), subfolder="IHC")


# RAPORT
print("\nPodsumowanie statystyczne (średnie po neighborhood_type):")
print(df.groupby("neighborhood_type")[["mean_cost", "min_cost", "mean_runtime"]]
      .mean().round(3))

best = df.loc[df["mean_cost"].idxmin()]
print(f"\nNajlepsze parametry:\n{best.to_dict()}")
# Minimalne wyniki dla sprawdzenia
# ---------- python
# 177_408.05
# 113_232.504
# 101_052.592
# 97_959.335
# ---------- excel
# 119_028.44
# 111_729.62
# @njit(cache=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Rozgrzewanie Numba (kompilacja JIT)...
Kompilacja zakończona.


Instancja: Dane_TSP_76.xlsx

Łączny czas eksperymentów: 0.93 min (55.80 sek)

Podsumowanie (pierwsze 20 wierszy):
        instance  n_starts  max_iter  stop_no_improve neighborhood_type  mean_cost  mean_runtime  min_cost                                                                                                                                                                                                                    min_route
Dane_TSP_76.xlsx      2000     20000             2000           two_opt  98734.925      9.290176 97924.329 35-32-33-34-38-39-40-41-60-59-61-62-58-57-63-64-73-72-71-65-66-50-49-51-56-55-52-53-54-42-43-28-27-26-29-30-31-19-20-5-10-9-8-7-6-4-3-2-75-76-1-23-22-21-25-24-46-45-44-48-47-69-68-67-70-0-74-14-13-12-11-15-16-17-18-37-36

Podsumowanie statystyczne (średnie po neighborhood_type):
                   